In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install mat4py
!pip install geneticalgorithm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for func-timeout: filename=func_timeout-4.3.5-py3-none-any.whl size=15077 sha256=e114c84a6828e5f0bf5b3c720203ce77a02b6eccb84925cb32f30cff044807aa
  Stored in directory: /root/.cache/pip/wheels/3f/83/19/b5552bb9630e353f7c5b15be44bf10900afe1abbbfcf536afd
Successfully built func-timeout


In [ ]:


root_path = 'drive/My Drive/TU/Master/MEP/Paper/Working_file_copied_from_final_code/Experiment_3+2b'
import sys, os
#path_dir = os.getcwd()
sys.path.append(root_path+"/Functions")

from general_functions import *
from muscle_functions import *
from integrator_functions import *
from matrices_functions import * #can make specific later if bloated
from constants_functions import *
from plot_functions import *
from save_functions import *

In [ ]:
#pick a trajectory number from Flash experiments, recall optimal trajectory data and store in optimal_trajectory vector
traj_nr = 3
testref_traj1 = mat4py.loadmat('drive/My Drive/TU/Master/MEP/Paper/Working_file_copied_from_final_code/Experiment_3+2b/Optimal_trajectories/ref_traj3.mat')
optimal_trajectories = np.array(testref_traj1['ref_traj'])

In [ ]:
#The stiffness region that the muscles start in must be in the experiment name. Names are:
#mostpos,verypos,newpos,newneg,mostneg for stiffness region 1-5. Region name must be preceded by stiff_start_
experiment_name = 'NEW_OKT_feedback_stiff_start_verypos_'

#protocol indicates the system configuration. 'straight_up_delay' for experiment 2 perturbation test, 'reaching' for experiment 3 robustness test
protocol = 'reaching'#'stroeve','straight_up','straight_up_delay','reaching'

save = True#If true, the experiment data will be saved to a dataset in /Results/data
Fmax_muscles = False  #keep False, this is set manually in the constants_functions function

#model type determines whether Huxley or Hill functions are used, v_type determines the used vce equation in the Huxley model (old is used) and FL_.. determines the used FL model (hill is used)
model_type,v_type,FL_type='hill','v_old','FL_hill' #'huxley','hill'---'v_old','v_new'---'FL_0','FL_hill','FL_huxley'
hill_type = 'thijs'#determines the hill model used. 'thijs' for hill_expanded, 'stroeve' for hill_simple
f_type = 'f_damped_0grav'#determines how the force vector in matrices_functions.py is calculated. 'f_damped_fullgrav','f_damped__pgrav','f_damped__0grav','f_undamped_fullgrav','f_undamped__pgrav','f_undamped__0grav' ------------apply manually to matrices_functions.py
straight_og=False #If True, the current_muscle_starting_lengths will be defined with the arm in the straight up position
add_feedback = True #If True, the feedback system will be used to change the STIM parameter


#some experiment parameters
m3_value = 0.00000001       #allows for manual scaling of ball mass and inertia
f_factor = 1.0              #alllows for manual scaling of muscle force
t0 = 0.0                    #time start point (s)
tf = 0.35                   #time end point   (s)
t_stimulation = 0.01        #s at this time, the muscle stimulation will start
t_pertubation = False       #s at this time, the system perturbation will be applied. If False, no perturbation
h_step = 0.001              #stepsize         (s) #can not change, optimal lengths have the same size
muscle_number = 6           #nr of muscles used
tot_bridge_nr = 5000        #number of cross-bridges in the Huxley Model
step_nr = int((tf-t0)/h_step)


#determines the initial excitation
stim_multiplier = 0.25#stim_multiplier_list[multiplier_nr]


best_hill1_vector = [ 1.79588357e+02,  1.48266795e+01 ,-9.51596047e-01 ,-2.74214174e-02]
best_hux_vector =  [99.25994797 , 8.53835633 ,-1.24112202, 0.10921817]

# the opt_hux_feedback_vector is the vector used for the robustness data
opt_hux_feedback_vector =best_hill1_vector


#make a vector of multipliers, to explore parameter space
fbvector_multiplier_list = np.linspace(0.75,1.25,21)#[0.85,0.90,0.95,1.00,1.05,1.10,1.15]
'''
 #uncomment this, uncomment the last comment section with the breaks, and add a tab to all lines in between these 2 commented sections in order to test over multiple initial vectors
bigloopcounter = 0
quit = False
lowest_traj_error=4.802153955808967#2.7883803446089366#3.0960973744579343#0.007245737025702425 #fill in accurately
while quit == False:
  quit = True
  bigloopcounter += 1
  breakvar = 0
  print('Big loop nr: '+str(bigloopcounter))
  print('current lowest traj error: ')
  print(lowest_traj_error)
  print('Current optimal feedback vector: ')
  print(opt_hux_feedback_vector)
'''
stored_fbvector_multipliers = np.zeros((len(opt_hux_feedback_vector),len(fbvector_multiplier_list)))
stored_traj_errors = np.zeros((len(opt_hux_feedback_vector),len(fbvector_multiplier_list)))
feedback_vector = 0
for fb_var_nr in range(len(opt_hux_feedback_vector)):
    for fbvector_multiplier_nr in range(len(fbvector_multiplier_list)):
        mus_vals = muscle_vals_func()
        feedback_vector = np.array(opt_hux_feedback_vector)
        fbvector_multiplier = np.array([1,1,1,1])
        multiplier = fbvector_multiplier_list[fbvector_multiplier_nr]
        #fbvector_multiplier[fb_var_nr] *= multiplier
        feedback_vector[fb_var_nr] = opt_hux_feedback_vector[fb_var_nr]*multiplier

        print(feedback_vector)



        stored_fbvector_multipliers[fb_var_nr][fbvector_multiplier_nr] = multiplier
        #fbvector_multiplier = fbvector_multiplier_list[fbvector_multiplier_nr]


        if model_type == 'huxley':
            model_name = 'huxley'
        if model_type == 'hill':
            model_name = hill_type

        specific_save_name = str(model_name)+'_fbvec_'+str(opt_hux_feedback_vector[:])+'_scaled_'+str(fbvector_multiplier)+'_stimmult'+str(stim_multiplier)

        if protocol == 'straight_up':
            starting_a1 = 0.5*sm.pi
        if protocol == 'straight_up_delay':
            starting_a1 = 0.5*sm.pi
        if protocol == 'stroeve':
            starting_a1 = 0.2*sm.pi
        if protocol == 'reaching':
            starting_a1 = 0.5*sm.pi


        l1, l2,l3,l4,m1, m2, J1, J2,F_mtc1,F_mtc2,F_mtc3,F_mtc4,F_mtc5,F_mtc6,r1,r2,B1,B2,thetar1,thetar2,PE_sh1,PE_sh2,PE_xm1,PE_xm2,T_max1,T_max2 = sm.symbols('l_1,l_2,l_3,l_4, m_1, m_2, J_1, J_2, F_{mtc_1},F_{mtc_2},F_{mtc_3},F_{mtc_4},F_{mtc_5},F_{mtc_6},r_1,r_2,B_1,B_2,theta_{r1},theta_{r2},PE_{sh1},PE_{sh2},PE_{xm1},PE_{xm2},T_{max1},T_{max2}')
        t = sm.symbols('t')
        alpha1 = sm.Function('alpha1')(t)
        alpha2 = sm.Function('alpha2')(t)
        alpha3 = sm.Function('alpha3')(t)
        J3, m3 = sm.symbols('J_3,m_3')

        #The joint locations and body center of masses are defined
        x0=0
        y0=0
        x1=0.5*l1*(sm.cos(alpha1))
        y1=0.5*l1*(sm.sin(alpha1))
        x2=2*x1
        y2=2*y1
        x3=x2+0.5*l2*(sm.cos(alpha2))#alpha1 +
        y3=y2+0.5*l2*(sm.sin(alpha2))#alpha1 +
        x4=x2+l2*(sm.cos(alpha2))#alpha1 +
        y4=y2+l2*(sm.sin(alpha2))#alpha1 +

        #Here the starting and ending locations of the muscles are defined, which are later included in the state vector
        #this makes the starts and endings of the muscles scale directly with the joint angle
        #start is defined as the lowest point in the original stroeve image
        xm1_start=-sm.sin(starting_a1)*r1 #-sm.sin(alpha1)*r1
        ym1_start=sm.cos(starting_a1)*r1  #sm.cos(alpha1)*r1
        xm1_end  =x2*l3/l1
        ym1_end  =y2*l3/l1
        xm2_start=sm.sin(starting_a1)*r1#sm.sin(alpha1)*r1
        ym2_start=-sm.cos(starting_a1)*r1#-sm.cos(alpha1)*r1
        xm2_end  =x2*l3/l1
        ym2_end  =y2*l3/l1
        xm3_start=x2*(l1-l4)/l1
        ym3_start=y2*(l1-l4)/l1
        xm3_end  =x2-sm.sin(alpha2)*r1#alpha1 +
        ym3_end  =y2+sm.cos(alpha2)*r1#alpha1 +
        xm4_start=x2*(l1-l4)/l1
        ym4_start=y2*(l1-l4)/l1
        xm4_end  =x2+sm.sin(alpha2)*r1#alpha1 +
        ym4_end  =y2-sm.cos(alpha2)*r1#alpha1 +
        xm5_start=-sm.sin(starting_a1)*r1
        ym5_start=sm.cos(starting_a1)*r1
        xm5_end  =xm3_end
        ym5_end  =ym3_end
        xm6_start=sm.sin(starting_a1)*r1
        ym6_start=-sm.cos(starting_a1)*r1
        xm6_end  =xm4_end
        ym6_end  =ym4_end

        #I might need to redefine these to simply increase or decrease the length in a pulley-like fashion
        l_mtc1=sm.sqrt((xm1_end-xm1_start)**2+(ym1_end-ym1_start)**2)
        l_mtc2=sm.sqrt((xm2_end-xm2_start)**2+(ym2_end-ym2_start)**2)
        l_mtc3=sm.sqrt((xm3_end-xm3_start)**2+(ym3_end-ym3_start)**2)
        l_mtc4=sm.sqrt((xm4_end-xm4_start)**2+(ym4_end-ym4_start)**2)
        l_mtc5=sm.sqrt((xm5_end-xm5_start)**2+(ym5_end-ym5_start)**2)
        l_mtc6=sm.sqrt((xm6_end-xm6_start)**2+(ym6_end-ym6_start)**2)

        x = sm.Matrix([alpha1, x1, y1,alpha2,x3,y3,alpha3,x4,y4])
        q = sm.Matrix([alpha1,alpha2,alpha3])
        T_mat = x.diff(t).jacobian(q.diff(t))

        #define constants and specifieds
        p_protocol_stroeve = {
            l1: 0.32,    # m
            l2: 0.32,  # m
            l3: 0.125,  # m
            l4: 0.125,    #m
            m1: 1.8,    # kg
            J1: 0.015,    # kg*m^2 https://aip-scitation-org.tudelft.idm.oclc.org/doi/pdf/10.1063/1.3479844#:~:text=We%20can%20see%20that%20the,0.1kgm2%20to%200.5kgm2.
            m2: 1.6,    # m
            J2: 0.013,    # kg*m^2
            r1: 0.03,   #rad
            r2: 0.03,   #rad
            m3: 0.00000001,
            J3: 0.0000001, #ball: 2/5*M*r2. Say r of 0.05m, m of 2: 0.002
            B1: 0.3,#Shoulder damping, Nms/rad
            B2: 0.2,#Elbow damping, Nms/rad
            thetar1: 0.2*np.pi,#Shoulder rest angle, rad np.pi/3 OG
            thetar2: 0.2*np.pi,#Elbow rest angle, rad np.pi/2 OG
            PE_sh1: 18,#Shoulder PE shape
            PE_sh2: 18,#Elbow PE shape
            PE_xm1: 0.5*np.pi,#Shoulder PE range, rad
            PE_xm2: 0.5*np.pi,#Elbow PE range, rad
            T_max1: 75,#maximum shoulder torque, winters and stark 85
            T_max2: 51,#maximum elbow torque, winters and stark 85
        }

        p_protocol_straight_up = {
            l1: 0.32,    # m
            l2: 0.32,  # m
            l3: 0.125,  # m
            l4: 0.125,    #m
            m1: 1.8,    # kg
            J1: 0.015,    # kg*m^2 https://aip-scitation-org.tudelft.idm.oclc.org/doi/pdf/10.1063/1.3479844#:~:text=We%20can%20see%20that%20the,0.1kgm2%20to%200.5kgm2.
            m2: 1.6,    # m
            J2: 0.013,    # kg*m^2
            r1: 0.03,   #rad
            r2: 0.03,   #rad
            m3: m3_value,#0.00000001,#
            J3: 0.0008*m3_value, #ball: 2/5*M*r^2. Say r of 0.04m, m of 1: 0.00064. Circle: 0.5*m*r^2v #0.00000001,
            B1: 0.3,#Shoulder damping, Nms/rad
            B2: 0.2,#Elbow damping, Nms/rad
            thetar1: 1/3*np.pi,#Shoulder rest angle, rad
            thetar2: 0.5*np.pi,#Elbow rest angle, rad
            PE_sh1: 18,#Shoulder PE shape
            PE_sh2: 18,#Elbow PE shape
            PE_xm1: 0.5*np.pi,#Shoulder PE range, rad
            PE_xm2: 0.5*np.pi,#Elbow PE range, rad
            T_max1: 75,#maximum shoulder torque, winters and stark 85
            T_max2: 51,#maximum elbow torque, winters and stark 85
        }
        #initial conditions
        init_protocol_stroeve = {
            alpha1: 0.2*np.pi,
            alpha2: 0.2*np.pi,
            alpha3: 0.0*np.pi,
            alpha1.diff(t): 0,
            alpha2.diff(t): 0,
            alpha3.diff(t): 0,
        }
        init_protocol_straight_up = {
            alpha1: 0.500000*np.pi,
            alpha2: 0.50000*np.pi,
            alpha3: 0.0*np.pi,
            alpha1.diff(t): 0,
            alpha2.diff(t): 0.001*np.pi,
            alpha3.diff(t): 0,
        }
        init_protocol_straight_up_delay = {
            alpha1: 0.50*np.pi,
            alpha2: 0.50*np.pi,
            alpha3: 0.0*np.pi,
            alpha1.diff(t): 0.0*np.pi,
            alpha2.diff(t): 0.0*np.pi,
            alpha3.diff(t): 0.0*np.pi,
        }
        init_protocol_reaching = {
            alpha1: 0.2086*np.pi,
            alpha2: 0.7914*np.pi,
            alpha3: 0.0000000001*np.pi,
            alpha1.diff(t): 0.0*np.pi,
            alpha2.diff(t): 0.0*np.pi,
            alpha3.diff(t): 0.0*np.pi,
        }
        pert_protocol_straight_up_delay = {
            alpha1: 0.0*np.pi,
            alpha2: 0.0*np.pi,
            alpha3: 0.0*np.pi,
            alpha1.diff(t): 0.0*np.pi,
            alpha2.diff(t): 0.005*np.pi,
            alpha3.diff(t): 0.0*np.pi,
        }

        if protocol == 'straight_up':
            init = init_protocol_straight_up
            p = p_protocol_straight_up
            t_pertubation = False
            pert_vals = False
        if protocol == 'stroeve':
            init = init_protocol_stroeve
            p = p_protocol_stroeve
            pert_vals = False
        if protocol == 'straight_up_delay':
            init = init_protocol_straight_up_delay
            p = p_protocol_straight_up
            pert_vals = np.array(list(pert_protocol_straight_up_delay.values()))
        if protocol == 'reaching':
            init = init_protocol_reaching
            p = p_protocol_straight_up
            t_pertubation = False
            pert_vals = False

        r = {
            F_mtc1:   0.000000001,  # N
            F_mtc2:   0.000000001,  # N
            F_mtc3:   0.000000001,  # N
            F_mtc4:   0.000000001,  # N
            F_mtc5:   0.000000001,  # N
            F_mtc6:   0.000000001,  # N
        }

        #invoke system calculation functions
        F_newton, M_newton, q_args, qd_args, p_args, r_args = F_M_vars_calc(l1, l2,l3,l4,m1, m2, J1, J2,F_mtc1,F_mtc2,F_mtc3,F_mtc4,F_mtc5,F_mtc6,r1,r2,t,alpha1,alpha2,x0,x1,x2,x3,x4,y0,y1,y2,y3,y4,p,r,J3,m3,alpha3,B1,B2,thetar1,thetar2,PE_sh1,PE_sh2,PE_xm1,PE_xm2,T_max1,T_max2,f_type);
        M_tmt, Q_tmt, C_M, C_Q, A, b, eval_A, eval_b, eval_C, eval_Cd, eval_Cq, g_conv = matrices_func(M_newton, F_newton, q_args, qd_args, p_args, r_args, T_mat,x,q,t);
        init_vals = np.array(list(init.values()))

        #Make arrays of the values in order for use in the numerical functions
        q_vals = np.array([init_vals[0],init_vals[1],init_vals[2]])
        qd_vals = np.array([init_vals[3], init_vals[4], init_vals[5]])
        p_vals = np.array([p[p_sym] for p_sym in p_args])
        r_vals = np.array([r[r_sym] for r_sym in r_args])
        eval_xs = sm.lambdify((q_args, p_args), [x0,x0,x1,x2,x3,x4])
        eval_ys = sm.lambdify((q_args, p_args), [y0,y0,y1,y2,y3,y4])
        eval_ls = sm.lambdify((q_args, p_args), [l_mtc1, l_mtc2,l_mtc3,l_mtc4,l_mtc5,l_mtc6])

        #make an input stimulation vector
        step_nr = int((tf-t0)/h_step)
        input_STIM = [[]]*muscle_number
        starting_step = int((t_stimulation/tf)*step_nr)
        input_STIM[0] = np.ones(step_nr+2)*stim_multiplier*1
        input_STIM[1] = np.ones(step_nr+2)*stim_multiplier*1
        input_STIM[2] = np.ones(step_nr+2)*stim_multiplier*1
        input_STIM[3] = np.ones(step_nr+2)*stim_multiplier*1
        input_STIM[4] = np.ones(step_nr+2)*stim_multiplier*1
        input_STIM[5] = np.ones(step_nr+2)*stim_multiplier*1
        input_STIM[0][0:starting_step] = 0 #this way, step input should be applied at 0.01 s
        input_STIM[1][0:starting_step] = 0
        input_STIM[2][0:starting_step] = 0
        input_STIM[3][0:starting_step] = 0
        input_STIM[4][0:starting_step] = 0
        input_STIM[5][0:starting_step] = 0

        #create function to iterate over with rk4
        eval_rhs2 = lambda t, s, r_vals: eval_rhs(t, s, p_vals, r_vals,eval_A, eval_b)[0]  # don't output the Lagrange multipliers

        for kkk in range(muscle_number):#scale the maximum force variables
            mus_vals[kkk][7]*=f_factor

        #execute calculations
        t_vals, s_vals, coords_vals,mtc_vels,calculated_muscle_forces,states,activation,mus_vals_changed,K_SE_list,K_CE_list,force_vector = rk4_integrator(eval_rhs2, t0, tf, init_vals, t_pertubation,pert_vals,h_step, r_vals, muscle_number, tot_bridge_nr, eval_xs, eval_ys, eval_ls,p_vals,mus_vals,Fmax_muscles,  f_factor,False,True,input_STIM,True,False,model_type,hill_type,v_type,FL_type,straight_og,specific_save_name,experiment_name,optimal_trajectories,feedback_vector,add_feedback)

        #some data preparation
        l_mtcs = [[]]*muscle_number
        for muscle in range(muscle_number):
            l_mtcs[muscle] = coords_vals[:,2][:,muscle]
        if model_type =='huxley':
            lCEs = np.zeros((muscle_number,len(states[0][:,-2])))
            lSEs = np.zeros((muscle_number,len(states[0][:,-2])))
            for i in range(muscle_number):
                lCEs[i] = states[i][:,-2]
            for i in range(muscle_number):
                lSEs[i] = coords_vals[:,2,i] - lCEs[i]
        if model_type =='hill':
            if hill_type == 'stroeve':
                lCEs = np.zeros((muscle_number,len(states[0])))
                vCEs = np.zeros((muscle_number,len(states[0])))
                for muscle in range(muscle_number):
                    lCEs[muscle] = l_mtcs[muscle] - 0.02
                    vCEs[muscle] = np.gradient(lCEs[muscle],h_step)
            if hill_type == 'thijs':
                lCEs = np.zeros((muscle_number,len(states[0])))
                vCEs = np.zeros((muscle_number,len(states[0])))
                for muscle in range(muscle_number):
                    lCEs[muscle] = states[muscle][:,0]
                    vCEs[muscle] = force_vector[muscle][:,7]
        optimal_muscle_length_array = np.zeros((len(t_vals),muscle_number))
        for a in range(len(t_vals)):
            if a <=750: #length of optimal value vector
                optimal_muscle_length_array[a] = eval_ls([optimal_trajectories[:,1][a-1],optimal_trajectories[:,2][a-1]+optimal_trajectories[:,1][a-1],0],p_vals)#determine optimal muscle lengths at this time
            if a >750: #length of optimal value vector
                optimal_muscle_length_array[a] = eval_ls([optimal_trajectories[:,1][750],optimal_trajectories[:,2][750]+optimal_trajectories[:,1][750],0],p_vals)#determine optimal muscle lengths at this time

                    #save data
        # if save == True:
        #     if model_type =='huxley':
        #         save_huxley(l_mtcs,tot_bridge_nr,states,force_vector,mtc_vels,input_STIM,lCEs,lSEs,K_CE_list,K_SE_list,coords_vals,s_vals,t_vals,experiment_name, specific_save_name)
        #     if model_type =='hill':
        #         save_hill(l_mtcs,calculated_muscle_forces,force_vector,mtc_vels,input_STIM,lCEs,vCEs,t_vals,s_vals,experiment_name,specific_save_name)


        # if model_type =='huuxley':
        #     plot_huxley_single_muscle(t_vals, s_vals, coords_vals,mtc_vels,calculated_muscle_forces,states,input_STIM,t0,tf,t_pertubation,h_step,muscle_number,tot_bridge_nr,step_nr,mus_vals_changed,np.array(list(p.values())),v_type,FL_type,f_type,K_SE_list,K_CE_list,lCEs,lSEs,save,experiment_name,specific_save_name)


        # #plot data
        # if model_type =='huxley':
        #     plot_huxley_full(t_vals, s_vals, coords_vals,mtc_vels,calculated_muscle_forces,states,input_STIM,t0,tf,t_pertubation,h_step,muscle_number,tot_bridge_nr,step_nr,mus_vals_changed,np.array(list(p.values())),v_type,FL_type,f_type,K_SE_list,K_CE_list,lCEs,lSEs,save,experiment_name,specific_save_name,optimal_trajectories,optimal_muscle_length_array)
        # if model_type == 'hill':
        #     plot_hill_full(t_vals, s_vals, coords_vals,mtc_vels,calculated_muscle_forces,states,input_STIM,t0,tf,t_pertubation,h_step,muscle_number,tot_bridge_nr,step_nr,mus_vals,save,experiment_name,specific_save_name,optimal_trajectories,optimal_muscle_length_array)

        opt_a1 = np.ones(len(t_vals))*optimal_trajectories[-1,1]
        opt_a2 = np.ones(len(t_vals))*(optimal_trajectories[-1,2]+optimal_trajectories[-1,1])
        if int(tf/h_step) > len(optimal_trajectories[:,1]):
            opt_a1[:len(optimal_trajectories[:,1])] = optimal_trajectories[:,1]
            opt_a2[:len(optimal_trajectories[:,2])] = optimal_trajectories[:,2]+optimal_trajectories[:,1]
        if int(tf/h_step) <= len(optimal_trajectories[:,1]):
            opt_a1[:int(tf/h_step)] = optimal_trajectories[:int(tf/h_step),1]
            opt_a2[:int(tf/h_step)] = optimal_trajectories[:int(tf/h_step),2]+optimal_trajectories[:int(tf/h_step),1]


        plt.close()

        '''
        fig, axes = plt.subplots(2,figsize=(4,10))

        axes[0].plot(t_vals,s_vals[:,0],label='a1')
        axes[0].plot(t_vals,s_vals[:,1],label='a2')
        axes[0].plot(t_vals,opt_a1,label='a1 optimal')
        axes[0].plot(t_vals,opt_a2,label='a2 optimal')
        axes[0].set_xlabel('time [s]')
        axes[0].set_ylabel('joint angle [rad]')
        axes[0].legend(loc='right')
        axes[0].set_title('Simulated vs optimal joint angles')
        len(coords_vals[1])
        opt_x = eval_xs([opt_a1,opt_a2,0],p_vals)#determine optimal muscle lengths at this time
        opt_y = eval_ys([opt_a1,opt_a2,0],p_vals)
        #print(opt_x)
        #integrated_trajectory_error = np.trapz((s_vals[:,1]-opt_a2)**2)+2*np.trapz((s_vals[:,0]-opt_a1)**2)
        integrated_trajectory_error = np.trapz((np.sqrt(opt_x[-1]**2+opt_y[-1]**2)-np.sqrt(coords_vals[:,1,-1]**2+coords_vals[:,0,-1]**2))**2)
        axes[1].plot(t_vals,coords_vals[:,0,-1],label='x4')
        axes[1].plot(t_vals,coords_vals[:,1,-1],label='y4')
        axes[1].plot(t_vals,opt_x[-1],label='x4 optimal')
        axes[1].plot(t_vals,opt_y[-1],label='y4 optimal')
        axes[1].set_xlabel('time [s]')
        axes[1].set_ylabel('length [m]')
        axes[1].legend(loc='right')
        axes[1].set_title('Simulated vs optimal x4 coordinates')
        #fig.savefig('Results/sm_plot_'+str(experiment_name)+'__'+str(specific_save_name)+'.png')
        stored_traj_errors[fb_var_nr][fbvector_multiplier_nr] = integrated_trajectory_error
        print(integrated_trajectory_error)
        '''
        opt_x = eval_xs([opt_a1,opt_a2,0],p_vals)#determine optimal muscle lengths at this time
        opt_y = eval_ys([opt_a1,opt_a2,0],p_vals)
        #integrated_trajectory_error = np.trapz((np.sqrt(opt_x[-1]**2+opt_y[-1]**2)-np.sqrt(coords_vals[:,1,-1]**2+coords_vals[:,0,-1]**2))**2)
        integrated_trajectory_error2 = np.trapz(np.sqrt((opt_x[-1]-coords_vals[:,0,-1])**2+(opt_y[-1]-coords_vals[:,1,-1])**2))
        integrated_trajectory_error = integrated_trajectory_error2
        stored_traj_errors[fb_var_nr][fbvector_multiplier_nr] = integrated_trajectory_error
        print(integrated_trajectory_error2)

'''
          if integrated_trajectory_error < lowest_traj_error:
            print('New lowest traj error found: ')
            print(integrated_trajectory_error)
            print('New associated feedback vector: ')
            print(feedback_vector)
            print('')
            lowest_traj_error=integrated_trajectory_error
            opt_hux_feedback_vector = feedback_vector
            breakvar = 1
            quit = False
            break
      if breakvar == 1:
        break
        '''


[ 1.34691268e+02  1.48266795e+01 -9.51596047e-01 -2.74214174e-02]


/content/drive/My Drive/TU/Master/MEP/Paper/Working_file_copied_from_final_code/Experiment_3+2b/Functions/muscle_functions.py:492: RuntimeWarning: invalid value encountered in scalar divide
  f_rel = activation*F_SE_prev/(Fmax*FL_prev)#F_SE_prev/(Fmax*activation*FL_prev)


4.021759103433965
[ 1.39180977e+02  1.48266795e+01 -9.51596047e-01 -2.74214174e-02]
3.9887679989112304
[ 1.43670686e+02  1.48266795e+01 -9.51596047e-01 -2.74214174e-02]
3.957171434974309
[ 1.48160395e+02  1.48266795e+01 -9.51596047e-01 -2.74214174e-02]
3.9267630584043713
[ 1.52650103e+02  1.48266795e+01 -9.51596047e-01 -2.74214174e-02]
3.8969659904454215
[ 1.57139812e+02  1.48266795e+01 -9.51596047e-01 -2.74214174e-02]
3.8673112019247045
[ 1.61629521e+02  1.48266795e+01 -9.51596047e-01 -2.74214174e-02]
3.8396038530323775
[ 1.66119230e+02  1.48266795e+01 -9.51596047e-01 -2.74214174e-02]
3.8145878553518022
[ 1.70608939e+02  1.48266795e+01 -9.51596047e-01 -2.74214174e-02]
3.7930725740798312
[ 1.75098648e+02  1.48266795e+01 -9.51596047e-01 -2.74214174e-02]
3.776280796847777
[ 1.79588357e+02  1.48266795e+01 -9.51596047e-01 -2.74214174e-02]
3.7673798147495305
[ 1.84078066e+02  1.48266795e+01 -9.51596047e-01 -2.74214174e-02]
3.7694457609476646
[ 1.88567775e+02  1.48266795e+01 -9.51596047e-01 

"\n          if integrated_trajectory_error < lowest_traj_error:\n            print('New lowest traj error found: ')\n            print(integrated_trajectory_error)\n            print('New associated feedback vector: ')\n            print(feedback_vector)\n            print('')\n            lowest_traj_error=integrated_trajectory_error\n            opt_hux_feedback_vector = feedback_vector\n            breakvar = 1\n            quit = False\n            break\n      if breakvar == 1:\n        break\n        "

In [ ]:
print(lowest_traj_error)

In [ ]:
#print(lowest_traj_error)#[30.85100035 12.69353107 -1.23699869  0.10921817]

traj_error_data = xr.Dataset(
    data_vars=dict(
        integrated_trajectory_errors = (['fbvar_nr','multiplier'],stored_traj_errors),
        multiplier_values = (['fbvar_nr','multiplier'],stored_fbvector_multipliers),
    ),
    attrs=dict(description="Integrated trajectory errors of simulated scaled optimal trajectories"),
    )

traj_error_data.to_netcdf('drive/My Drive/TU/Master/MEP/Paper/Working_file_copied_from_final_code/Experiment_3+2b/Results/data/NEW_OKT_'+str(model_name)+'traj'+str(traj_nr)+'traj_error_data_ogfbvec_'+str(opt_hux_feedback_vector[:])+'THIS')


In [ ]:
traj_error_data.integrated_trajectory_errors[:].min()